In [ ]:
import keras
from keras import backend as K
from keras.models import Sequential, Model
from tensorflow.contrib.keras import layers
from keras.layers import Input, LSTM, RepeatVector
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
from keras import objectives
from keras import regularizers
from sklearn.linear_model import Lasso

data = np.expand_dims(sub_data, axis=1)
data = np.reshape(data, (793*7,405 , 59421))
latent_dim = [32]

input_dim = data.shape[-1] # 13
timesteps =  data.shape[1]# 3

inputs = Input(shape=(timesteps, input_dim,))
layer1 = Dense(128, activation='tanh', activity_regularizer=regularizers.l1(1*10e-7), kernel_regularizer=regularizers.l2(1*10e-4))#
encoded = layer1(inputs)
encoded = LSTM(64,return_sequences=True, activation='tanh', activity_regularizer=regularizers.l1(1*10e-7), kernel_regularizer=regularizers.l2(1*10e-4))(encoded)
encoded = LSTM(32,return_sequences=True, activation='tanh', activity_regularizer=regularizers.l1(1*10e-7), kernel_regularizer=regularizers.l2(1*10e-4))(encoded)

encoder = Model(inputs, encoded)
encoder.summary()

inputs = Input(shape=(timesteps, 32,))
decoded = LSTM(64, return_sequences=True,activation='tanh')(inputs)
decoded = LSTM(128, return_sequences=True,activation='tanh')(decoded)
outputs = Dense(input_dim, activation='tanh')(decoded)


decoder = Model(inputs, outputs)
decoder.summary()
# model for RAE
inputs = Input(shape=(timesteps, input_dim,))
outputs = encoder(inputs)
outputs = decoder(outputs)
sequence_autoencoder = Model(inputs, outputs)

# autoencoder = Model(inputs, decoded)

sequence_autoencoder.compile(optimizer='adam', loss='mse')
sequence_autoencoder.summary()
sequence_autoencoder.fit(data, data, epochs=1, batch_size =1)